--- Day 5: If You Give A Seed A Fertilizer ---

You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4

The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

50 98 2
52 50 48

The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51

With this map, you can look up the soil number required for each initial seed number:

    Seed number 79 corresponds to soil number 81.
    Seed number 14 corresponds to soil number 14.
    Seed number 55 corresponds to soil number 57.
    Seed number 13 corresponds to soil number 13.

The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

    Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
    Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
    Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
    Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.

So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [ ]:
file_input = open(file="./Inputs/Day5.txt", mode='r')
my_input = file_input.read()

In [ ]:
class Mapping:
    first_source : int
    first_dest : int
    length : int
    
    def __init__(self, first_source : int, first_dest : int, length : int):
        self.first_source = first_source
        self.first_dest = first_dest
        self.length = length
    
    def lastStart(self) -> int:
        return self.first_source + self.length - 1
    
    def difSourceDest(self):
        return self.first_dest - self.first_source
    
    def map(self, val) -> int:
        if val >= self.first_source and val <= self.lastStart():
            return val + self.difSourceDest()
        else:
            return val
    
class MappingSet:
    mappings : list[Mapping]
    
    def __init__(self):
        self.mappings = []
    
    def map(self, val : int):
        n_val = val
        for mapping in self.mappings:
            n_val = mapping.map(val)
            if n_val != val:
                break
        
        return n_val

paragraphs = my_input.split("\n\n")

initials_str = paragraphs[0].split(" ")
initials_str.pop(0)
paragraphs.pop(0)
initials : list[int] = []
for initial in initials_str:
    initials.append(int(initial))

mapping_sets : list[MappingSet] = []
for paragraph in paragraphs:
    ranges = paragraph.split("\n")
    ranges.pop(0)
    
    mapping_sets.append(MappingSet())
    for interval in ranges:
        values = interval.split(" ")
        first_source = int(values[1])
        first_dest = int(values[0])
        length = int(values[2])
        
        mapping_sets[-1].mappings.append(Mapping(first_source=first_source, first_dest=first_dest, length=length))

values = initials.copy()

for mapping_set in mapping_sets:
    for i, value in enumerate(values):
        values[i] = mapping_set.map(value)

values.sort()

print("Answer:")
print(values[0])

--- Part Two ---

Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:

seeds: 79 14 55 13

This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?

In [ ]:
file_input = open(file="./Inputs/Day5.txt", mode='r')
my_input = file_input.read()

class Range:
    first_ele : int
    size : int
    
    def __init__(self, first_ele : int, size : int):
        self.first_ele = first_ele
        self.size = size
    
    def __lt__(self, other):
        if self.first_ele < other.first_ele:
            return True
        elif self.first_ele == other.first_ele:
            return self.size < other.size
        
        return False
    
    def lastEle(self):
        return self.first_ele + self.size - 1
    
    def inRange(self, val):
        return val >= self.first_ele and val <= self.lastEle()

class Mapping:
    source : Range
    first_dest : int
    
    def __init__(self, first_source : int, first_dest : int, length : int):
        self.source = Range(first_source, length)
        self.first_dest = first_dest
    
    def difSourceDest(self):
        return self.first_dest - self.source.first_ele
    
    # Returns the oputput range and an int indicationg if a value has changed 
    # (-1 for no change, 0 for first intercal, 1 for the seccond, and 2 for the hole)
    def map(self, interval : Range) -> (list[Range], int):
        first_in_range = self.source.inRange(interval.first_ele)
        last_in_range = self.source.inRange(interval.lastEle())
        needs_to_split = first_in_range != last_in_range
        
        if needs_to_split:
            middle_point : int
            
            if first_in_range:
                middle_point = self.source.lastEle()
                resuting_list = [Range(interval.first_ele + self.difSourceDest(), middle_point - interval.first_ele + 1), \
                                 Range(middle_point+1, interval.lastEle() - middle_point)]
                return (resuting_list, 0)
            else:
                middle_point = self.source.first_ele
                resuting_list = [Range(interval.first_ele, middle_point - interval.first_ele), \
                                 Range(middle_point, interval.lastEle() - middle_point + 1)]
                return (resuting_list, 1)
            
        elif first_in_range:
            return ([Range(interval.first_ele + self.difSourceDest(), interval.size)], 2)

        else:
            return ([interval], -1)
            
        
    
class MappingSet:
    mappings : list[Mapping]
    
    def __init__(self):
        self.mappings = []
    
    def map(self, interval : Range) -> list[Range]:
        current_range = interval
        resulting_list = []
        
        for mapping in self.mappings:
            mapping_out = mapping.map(current_range)
            
            if mapping_out[1] == 2:
                current_range = mapping_out[0][0]
                break
            elif mapping_out[1] == 1:
                resulting_list.append(mapping_out[0][1])
                current_range = mapping_out[0][0]
            elif mapping_out[1] == 0:
                resulting_list.append(mapping_out[0][0])
                current_range = mapping_out[0][1]
        
        resulting_list.append(current_range)
        
        return resulting_list
                


paragraphs = my_input.split("\n\n")

initials_str = paragraphs[0].split(" ")
initials_str.pop(0)
paragraphs.pop(0)
values : list[Range] = []

for i in range(0, len(initials_str), 2):
    values.append(Range(int(initials_str[i]), int(initials_str[i+1])))

mapping_sets : list[MappingSet] = []
for paragraph in paragraphs:
    ranges = paragraph.split("\n")
    ranges.pop(0)
    
    mapping_sets.append(MappingSet())
    for range in ranges:
        range_data = range.split(" ")
        first_source = int(range_data[1])
        first_dest = int(range_data[0])
        length = int(range_data[2])
        
        mapping_sets[-1].mappings.append(Mapping(first_source=first_source, first_dest=first_dest, length=length))

for mapping_set in mapping_sets:
    next_values = []
    for value in values:
        next_values.extend(mapping_set.map(value))
    
    values = next_values

values.sort()

print("Answer:")
print(values[0].first_ele)